In [21]:
import os
from glob import glob
import random
from transformers import BartForConditionalGeneration, BartTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq
from torch.utils.data import Dataset, DataLoader
import datasets
import numpy as np
import chardet
import torch
from onnx2keras import onnx_to_keras
import onnx
import onnxruntime as ort


In [2]:
base_dir = "./News"
articles_dir = os.path.join(base_dir, "News Articles")
summaries_dir = os.path.join(base_dir, "Summaries")
categories = ["business", "entertainment","politics","sport","tech"]

In [3]:
def load_files(dir_path):
    all_files = []
    for category in categories:
        category_dir = os.path.join(dir_path, category)
        files = glob(os.path.join(category_dir,"*.txt"))
        all_files.extend(files)
    return all_files

In [4]:
articles = load_files(articles_dir)
summaries = load_files(summaries_dir)

In [5]:
data = []
for article_file, summary_file in zip(articles, summaries):
    with open(article_file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']
    with open(article_file, 'r', encoding=encoding) as f:
        article = f.read()
    with open(summary_file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']
    with open(summary_file, 'r', encoding=encoding) as f:
        summary = f.read()
    data.append((article, summary))

In [6]:
random.shuffle(data)
train_data = data[:int(0.8*len(data))]
val_data = data[int(0.8*len(data)):int(0.9*len(data))]
test_data = data[int(0.9*len(data)):]

In [7]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [8]:
def tokenize(text, tokenizer, max_length=1024):
    encoded = tokenizer.encode_plus(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']
    return input_ids, attention_mask

In [9]:
processed_data = []
for article, summary in data:
    input_ids, attention_mask = tokenize(article, tokenizer)
    target_ids, _ = tokenize(summary, tokenizer, max_length=128)
    processed_data.append({
        'input_ids': input_ids.squeeze(0),
        'attention_mask': attention_mask.squeeze(0),
        'labels': target_ids.squeeze(0)
    })


In [10]:
max_length = 512
def split_into_chunks(tensor, max_length=512, stride=256):
    chunks = []
    for i in range(0, tensor.size(0), stride):
        chunk = tensor[i:i + max_length]
        if chunk.size(0) < max_length:
            padding = torch.full((max_length - chunk.size(0),), tokenizer.pad_token_id, dtype=torch.long)
            chunk = torch.cat((chunk, padding), dim=0)
        chunks.append(chunk)
    return chunks

new_processed_data = []

for data_point in processed_data:
    input_ids = data_point['input_ids']
    attention_mask = data_point['attention_mask']
    labels = data_point['labels']
    if input_ids.size(0) > max_length:
        input_chunks = split_into_chunks(input_ids)
        mask_chunks = split_into_chunks(attention_mask)
        for input_chunk, mask_chunk in zip(input_chunks, mask_chunks):
            new_processed_data.append({
                'input_ids': input_chunk,
                'attention_mask': mask_chunk,
                'labels': labels
            })
    else:
        new_processed_data.append(data_point)

In [11]:
class SummarizationDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'input_ids': item['input_ids'],
            'attention_mask': item['attention_mask'],
            'labels': item['labels']
        }
    
train_dataset = SummarizationDataset(new_processed_data[:int(0.8 * len(new_processed_data))])
val_dataset = SummarizationDataset(new_processed_data[int(0.8 * len(new_processed_data)):int(0.9 * len(new_processed_data))])
test_dataset = SummarizationDataset(new_processed_data[int(0.9 * len(new_processed_data)):])
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
rouge = datasets.load_metric('rouge')
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {
        'rouge1': result['rouge1'].mid.fmeasure,
        'rouge2': result['rouge2'].mid.fmeasure,
        'rougeL': result['rougeL'].mid.fmeasure
    }

C:\Users\Admin\AppData\Local\Temp\ipykernel_16664\800427156.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric('rouge')
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True
)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [15]:
train_output = trainer.train()
print("Train output:", train_output)

  0%|          | 0/3560 [00:00<?, ?it/s]c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\data\data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
 14%|█▍        | 500/3560 [2:24:39<14:55:20, 17.56s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 2.868, 'grad_norm': 9.16285228729248, 'learning_rate': 1.7191011235955056e-05, 'epoch': 0.56}


 25%|██▌       | 890/3560 [4:22:34<13:51:50, 18.69s/it]c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                       
 25%|██▌       | 890/3560 [4:50:26<13:51:50, 18.69s/it]

{'eval_loss': 2.2086851596832275, 'eval_rouge1': 0.14401848773563136, 'eval_rouge2': 0.07378554002845934, 'eval_rougeL': 0.12266539776124694, 'eval_runtime': 1672.5973, 'eval_samples_per_second': 0.532, 'eval_steps_per_second': 0.067, 'epoch': 1.0}


 28%|██▊       | 1000/3560 [5:22:52<12:25:01, 17.46s/it] Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


...

{'loss': 2.0039, 'grad_norm': 3.84087872505188, 'learning_rate': 3.146067415730337e-06, 'epoch': 3.37}


 98%|█████████▊| 3500/3560 [18:24:43<17:11, 17.20s/it]  Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 1.9729, 'grad_norm': 3.815361976623535, 'learning_rate': 3.3707865168539325e-07, 'epoch': 3.93}


100%|██████████| 3560/3560 [18:42:07<00:00, 17.46s/it]c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                      
100%|██████████| 3560/3560 [19:08:44<00:00, 19.36s/it]

{'eval_loss': 2.134305953979492, 'eval_rouge1': 0.14758449181693845, 'eval_rouge2': 0.08042293515753574, 'eval_rougeL': 0.12762804765803415, 'eval_runtime': 1596.6955, 'eval_samples_per_second': 0.557, 'eval_steps_per_second': 0.07, 'epoch': 4.0}
{'train_runtime': 68924.4628, 'train_samples_per_second': 0.413, 'train_steps_per_second': 0.052, 'train_loss': 2.2356609301620654, 'epoch': 4.0}
Train output: TrainOutput(global_step=3560, training_loss=2.2356609301620654, metrics={'train_runtime': 68924.4628, 'train_samples_per_second': 0.413, 'train_steps_per_second': 0.052, 'total_flos': 8682647165337600.0, 'train_loss': 2.2356609301620654, 'epoch': 4.0})


In [20]:
model.save_pretrained('summary_model')
tokenizer.save_pretrained("summary_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [28]:
sample_input_ids = torch.tensor([tokenizer.encode("Sample input text", max_length=1024, padding="max_length", truncation=True)])
sample_attention_mask = torch.ones_like(sample_input_ids)
onnx_model_path = "summary_model.onnx"
torch.onnx.export(
    model,
    (sample_input_ids, sample_attention_mask),
    onnx_model_path,
    opset_version=14,
    input_names=['input_ids', 'attention_mask'],
    output_names=['output'],
    dynamic_axes={'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

print(f"Model zapisany w formacie ONNX do pliku: {onnx_model_path}")

Model zapisany w formacie ONNX do pliku: summary_model.onnx


In [31]:
eval_results = trainer.evaluate()
print("Eval results:", eval_results)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 112/112 [27:58<00:00, 14.98s/it]

Eval results: {'eval_loss': 2.134305953979492, 'eval_rouge1': 0.14758449181693845, 'eval_rouge2': 0.08042293515753574, 'eval_rougeL': 0.12762804765803415, 'eval_runtime': 1693.6747, 'eval_samples_per_second': 0.525, 'eval_steps_per_second': 0.066, 'epoch': 4.0}


In [32]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text
file_path = "./News/News Articles/business/001.txt"
text = read_file(file_path)
inputs = tokenizer(text, return_tensors="pt", max_length=1024, padding="max_length", truncation=True)
summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Skrócony tekst:")
print(summary_text)

Skrócony tekst:
The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high
